In [ ]:
!apt install swig cmake ffmpeg xvfb python3-opengl
!pip install stable-baselines3==2.0.0a5 gymnasium[box2d] huggingface_sb3 pyvirtualdisplay imageio[ffmpeg]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 swig4.0 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  libgle3 python3-numpy swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 python3-opengl swig swig4.0
  x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 14 newly installed, 0 to remove and 45 not upgraded.
Need to get 9,754 kB of archives.
After this operation, 25.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 freeglut3 amd64 2.8.1-6 [74.0 kB]
Get:2 http://

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gymnasium as gym
env = gym.make("BipedalWalker-v3", hardcore=True)
env.reset()

(array([ 2.7468193e-03,  4.6664100e-06, -6.0846622e-04, -1.6000031e-02,
         9.2301413e-02,  1.4125160e-03,  8.5999507e-01,  2.8001549e-04,
         1.0000000e+00,  3.2668311e-02,  1.4124634e-03,  8.5362875e-01,
        -9.6108561e-04,  1.0000000e+00,  4.4081357e-01,  4.4581968e-01,
         4.6142232e-01,  4.8954970e-01,  5.3410226e-01,  6.0246044e-01,
         7.0914817e-01,  8.8593090e-01,  9.3943083e-01,  1.0000000e+00],
       dtype=float32),
 {})

In [3]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

Observation Space Shape (24,)
Sample observation [-1.046032    1.4877827   0.2757145   1.3883171  -2.4427285   2.7782722
 -2.6016226  -2.080589    2.8476758  -0.84352136  1.4211878   0.23096603
 -1.0717239   4.1550703  -0.83793676 -0.28938302 -0.3662845  -0.906988
 -0.45381415 -0.37048042  0.834481   -0.7290059  -0.4268969  -0.58565193]


In [4]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.shape)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

Action Space Shape (4,)
Action Space Sample [-0.9444046  -0.24963161  0.12522684 -0.03218832]


In [5]:
from stable_baselines3.common.env_util import make_vec_env
env = make_vec_env('BipedalWalker-v3', n_envs=16)

In [6]:
from stable_baselines3 import A2C  # Change this line
model = A2C(  # Change this line
    policy = 'MlpPolicy',
    env = env,
    verbose=1
)

Using cpu device


In [7]:
from wasabi import Printer
import numpy as np
from stable_baselines3.common.base_class import BaseAlgorithm
from pathlib import Path
import tempfile
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecEnv,
    VecVideoRecorder,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
msg = Printer()

In [9]:
def generate_replay(
    model: BaseAlgorithm,
    eval_env: VecEnv,
    video_length: int,
    is_deterministic: bool,
    local_path: Path,
):
    """
    Generate a replay video of the agent
    :param model: trained model
    :param eval_env: environment used to evaluate the agent
    :param video_length: length of the video (in timesteps)
    :param is_deterministic: use deterministic or stochastic actions
    :param local_path: path of the local repository
    """
    # This is another temporary directory for video outputs
    # SB3 created a -step-0-to-... meta files as well as other
    # artifacts which we don't want in the repo.
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Step 1: Create the VecVideoRecorder
        env = VecVideoRecorder(
            eval_env,
            tmpdirname,
            record_video_trigger=lambda x: x == 0,
            video_length=video_length,
            name_prefix="",
        )

        obs = env.reset()
        lstm_states = None
        episode_starts = np.ones((env.num_envs,), dtype=bool)

        try:
            for _ in range(video_length):
                action, lstm_states = model.predict(
                    obs,
                    state=lstm_states,
                    episode_start=episode_starts,
                    deterministic=is_deterministic,
                )
                obs, _, episode_starts, _ = env.step(action)

            # Save the video
            env.close()

            # Convert the video with x264 codec
            inp = env.video_recorder.path
            out = local_path
            os.system(f"ffmpeg -y -i {inp} -vcodec h264 {out}".format(inp, out))
            print(f"Video saved to: {out}")
        except KeyboardInterrupt:
            pass
        except Exception as e:
            msg.fail(str(e))
            # Add a message for video
            msg.fail(
                "We are unable to generate a replay of your agent"
            )

In [10]:
import os
#create a directory to save the videos
video_dir = "/content/videos"
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

In [11]:
env_id = "BipedalWalker-v3"
# Train and generate video at every 10000 steps, adjust the timesteps to your liking
for i in range(0, 200000, 10000):
    model.learn(total_timesteps=10000)
    # Save the model
    model_name = "a2c-BipedalWalker-v3"
    model.save(model_name)
    video_name = f"replay_{i + 10000}.mp4"
    generate_replay(
        model=model,
        eval_env=DummyVecEnv([lambda: Monitor(gym.make(env_id, hardcore=True, render_mode="rgb_array"))]),
        video_length=1000,
        is_deterministic=True,
        local_path=os.path.join(video_dir, video_name)
    )

model_name = "a2c-BipedalWalker-v3"
model.save(model_name)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 120      |
|    ep_rew_mean        | -114     |
| time/                 |          |
|    fps                | 1693     |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -5.57    |
|    explained_variance | 0.835    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.23     |
|    std                | 0.975    |
|    value_loss         | 0.134    |
------------------------------------
Saving video to /tmp/tmpg49mh9wz/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpg49mh9wz/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpg49mh9wz/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpg49mh9wz/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_10000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 82.4     |
|    ep_rew_mean        | -110     |
| time/                 |          |
|    fps                | 1756     |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -5.49    |
|    explained_variance | -0.183   |
|    learning_rate      | 0.0007   |
|    n_updates          | 224      |
|    policy_loss        | -0.283   |
|    std                | 0.954    |
|    value_loss         | 0.0128   |
------------------------------------
Saving video to /tmp/tmppuloynzr/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmppuloynzr/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmppuloynzr/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmppuloynzr/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_20000.mp4
------------------------------------
| time/                 |          |
|    fps                | 2110     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -5.42    |
|    explained_variance | 0.656    |
|    learning_rate      | 0.0007   |
|    n_updates          | 349      |
|    policy_loss        | -0.123   |
|    std                | 0.938    |
|    value_loss         | 0.0181   |
------------------------------------
Saving video to /tmp/tmp38n_5j6f/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp38n_5j6f/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp38n_5j6f/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp38n_5j6f/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_30000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 75.7     |
|    ep_rew_mean        | -105     |
| time/                 |          |
|    fps                | 2160     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -5.34    |
|    explained_variance | 0.253    |
|    learning_rate      | 0.0007   |
|    n_updates          | 474      |
|    policy_loss        | -0.113   |
|    std                | 0.92     |
|    value_loss         | 0.0591   |
------------------------------------
Saving video to /tmp/tmpqq99brlq/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpqq99brlq/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpqq99brlq/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpqq99brlq/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_40000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 90.2     |
|    ep_rew_mean        | -110     |
| time/                 |          |
|    fps                | 2183     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -5.29    |
|    explained_variance | 0.659    |
|    learning_rate      | 0.0007   |
|    n_updates          | 599      |
|    policy_loss        | 0.166    |
|    std                | 0.909    |
|    value_loss         | 0.0223   |
------------------------------------
Saving video to /tmp/tmp20zn4h95/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp20zn4h95/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp20zn4h95/-step-0-to-step-1000.mp4



...

Moviepy - Done !
Moviepy - video ready /tmp/tmpzf98jpio/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_160000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 149      |
|    ep_rew_mean        | -100     |
| time/                 |          |
|    fps                | 2072     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -4.08    |
|    explained_variance | 0.915    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2099     |
|    policy_loss        | 0.271    |
|    std                | 0.672    |
|    value_loss         | 0.205    |
------------------------------------
Saving video to /tmp/tmpczqeuydf/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpczqeuydf/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpczqeuydf/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpczqeuydf/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_170000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 91.2     |
|    ep_rew_mean        | -108     |
| time/                 |          |
|    fps                | 2148     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -4.04    |
|    explained_variance | 0.773    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2224     |
|    policy_loss        | 0.193    |
|    std                | 0.664    |
|    value_loss         | 0.0821   |
------------------------------------
Saving video to /tmp/tmpqmtwi1m9/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpqmtwi1m9/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpqmtwi1m9/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpqmtwi1m9/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_180000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 125      |
|    ep_rew_mean        | -112     |
| time/                 |          |
|    fps                | 2071     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -3.98    |
|    explained_variance | 0.8      |
|    learning_rate      | 0.0007   |
|    n_updates          | 2349     |
|    policy_loss        | 0.15     |
|    std                | 0.656    |
|    value_loss         | 0.157    |
------------------------------------
Saving video to /tmp/tmpnhbwmne9/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpnhbwmne9/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpnhbwmne9/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpnhbwmne9/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_190000.mp4
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 287      |
|    ep_rew_mean        | -93.3    |
| time/                 |          |
|    fps                | 2168     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -3.92    |
|    explained_variance | 0.98     |
|    learning_rate      | 0.0007   |
|    n_updates          | 2474     |
|    policy_loss        | 0.259    |
|    std                | 0.647    |
|    value_loss         | 0.356    |
------------------------------------
Saving video to /tmp/tmpodlovgc2/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpodlovgc2/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpodlovgc2/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpodlovgc2/-step-0-to-step-1000.mp4
Video saved to: /content/videos/replay_200000.mp4


In [12]:
with open(os.path.join(video_dir, "filelist.txt"), "w") as f:
    for i in range(0, 200000, 10000):
        video_name = f"replay_{i + 10000}.mp4"
        f.write(f"file '{os.path.join(video_dir, video_name)}'\n")
# Concatenate all the videos into one
os.system(f"ffmpeg -f concat -safe 0 -i {os.path.join(video_dir, 'filelist.txt')} -c copy {os.path.join(video_dir, 'replay_all.mp4')}")

0

In [13]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('videos/replay_all.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [14]:
from stable_baselines3.common.evaluation import evaluate_policy
eval_env = Monitor(gym.make("BipedalWalker-v3"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=191.90 +/- 21.414620367749475
